<a href="https://colab.research.google.com/github/fezjo/ml-project/blob/master/ml_project_ciz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras import layers

vocab_size = 26*26

# vectorize layer adapt to bigrams

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
layers.TextVectorization(
    ngrams=(2,),
    output_sequence_length=???,)
)
layers.Embedding(
    input_dim=vocab_size,
    output_dim=???,) 
layers.Dropout(0.25)
3,4,5 -> layers.Conv2D(
layers.Activation(activations.selu)
layers.MaxPooling1D(
layers.Dense(300, activation=activations.softmax)
layers.Dense(authors)